In [1]:
#we load our libraries
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [2]:
! pip install pyreadstat

In [3]:
#we have one function that filters the ALLBUS data for whether respondents are german and of voting age
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [18]:
#we have another function that regroups the data into fitting categories to the data we have from the zensus
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])

    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [19]:
#and lastly we have a stratification function that takes the zensus data and the skewed dataset of allbus
#  and creates a stratified dataset based on the population shares of the zensus data
def stratifier(zensus_model,skewed_df):

    stratified_dataset=pd.DataFrame()
    samplesize_sum=0

    #we go through each row of the zensus groups
    for index, row in zensus_model.iterrows():
        #take in which share of the population this group represents
        samplesize = round(row["Share"] * 100)
        samplesize_sum += samplesize
        #and then we take a copy of the skewed allbus data
        sub_df = skewed_df.copy()
        
        for i in range(0, 4):
            #and then we filter it down, feature by feature of the speficic row of the zensus data
            sub_df = sub_df[sub_df[row.index[i]] == row.values[i]]
            print(row.index[i],row.values[i],len(sub_df))
        
        #we draw a sample from this group that correlates to the share of the population
        sampled_subgroup = sub_df.sample(samplesize, random_state=1, replace=True)
        #and add this sampled group to the overall stratified dataset
        stratified_dataset = pd.concat([stratified_dataset, sampled_subgroup])

    return stratified_dataset


In [20]:
#this is the zensus data, we remove all rows with NaN values
#as we need as detailed data as possible for the stratification
zensus_model=pd.read_csv("zensus_model.csv")
zensus_model=zensus_model.dropna()
zensus_model=zensus_model[zensus_model["Share"]>0.5]
zensus_model=zensus_model.sort_values(by="Share",ascending=False)
zensus_model

,Geschlecht,Besiedelung,OstWest,Alter,Zahl,Share
55,Weiblich,mittlere Besiedlungsdichte,West,Ü65,3676607.0,6.141327
53,Weiblich,mittlere Besiedlungsdichte,West,50-64,3328957.0,5.560620
25,Männlich,mittlere Besiedlungsdichte,West,50-64,3229345.0,5.394230
52,Weiblich,mittlere Besiedlungsdichte,West,30-49,2987387.0,4.990069
24,Männlich,mittlere Besiedlungsdichte,West,30-49,2946250.0,4.921354
27,Männlich,mittlere Besiedlungsdichte,West,Ü65,2897533.0,4.839978
39,Weiblich,dicht besiedelt,West,Ü65,2473001.0,4.130849
8,Männlich,dicht besiedelt,West,30-49,2348445.0,3.922793
36,Weiblich,dicht besiedelt,West,30-49,2334252.0,3.899086
37,Weiblich,dicht besiedelt,West,50-64,2144819.0,3.582661


In [21]:
#we read in the allbus data for 2021
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
# filter it for the german electorate

df_2021 = germanElectorate(df_2021)
# regroup it to fit the zensus data

df_2021= reformgroups(df_2021)
# and stratify it based on the zensus data
df_2021= stratifier(zensus_model,df_2021)
df_2021

Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter Ü65 223
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 50-64 247
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 50-64 233
Geschlecht Weiblich 2474
Besiedelung mittlere Besiedlungsdichte 1204
OstWest West 744
Alter 30-49 193
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter 30-49 176
Geschlecht Männlich 2419
Besiedelung mittlere Besiedlungsdichte 1182
OstWest West 752
Alter Ü65 263
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter Ü65 60
Geschlecht Männlich 2419
Besiedelung dicht besiedelt 451
OstWest West 247
Alter 30-49 79
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 30-49 83
Geschlecht Weiblich 2474
Besiedelung dicht besiedelt 465
OstWest West 250
Alter 50-64 60
Geschlecht Männlich 2419
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt21,eastwest,german,ep01,...,bik,gkpol,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
986,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),987.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;>499.999,100.000 - 499.999 E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3459,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3460.0,SIMULTAN,MAIL,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;<500.000,100.000 - 499.999 E.,ALTE BUNDESLAENDER,NaN,NaN,NaN,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1790,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1791.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,GUT,...,ZONE 2-4;>499.999,20.000 - 49.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3381,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),3382.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,Ü65,Weiblich,West,mittlere Besiedlungsdichte
4800,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),4801.0,SEQUENZIELL,CAWI,SPLIT C,ALTE BUNDESLAENDER,JA,NaN,...,ZONE 2-4;<500.000,5.000 - 19.999 EINW,ALTE BUNDESLAENDER,0.842875,ALTE BUNDESLAENDER,1.041708,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),2039.0,SIMULTAN,MAIL,SPLIT B,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,50-64,Männlich,Berlin,dicht besiedelt
1185,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),1186.0,SIMULTAN,MAIL,SPLIT A,ALTE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,ALTE BUNDESLAENDER,1.685751,ALTE BUNDESLAENDER,2.083416,50-64,Männlich,Berlin,dicht besiedelt
5268,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),5269.0,SIMULTAN,CAWI,SPLIT C,NEUE BUNDESLAENDER,JA,NaN,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,1.603923,NEUE BUNDESLAENDER,0.849389,50-64,Männlich,Berlin,dicht besiedelt
957,ALLBUS 2021,https://doi.org/10.4232/1.14238,2.0.1 (2023-12-13),958.0,SEQUENZIELL,CAWI,SPLIT A,NEUE BUNDESLAENDER,JA,TEILS TEILS,...,ZONE 1 ;>499.999,500.000 UND MEHR E.,NEUE BUNDESLAENDER,0.801962,NEUE BUNDESLAENDER,0.424695,50-64,Männlich,Berlin,dicht besiedelt


In [22]:
#we read in the allbus data for 2023
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
# filter it for the german electorate

df_2023 = germanElectorate(df_2023)
# regroup it to fit the zensus data

df_2023= reformgroups(df_2023)
#and straitify it based on the zensus data
df_2023= stratifier(zensus_model,df_2023)
df_2023


Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter Ü65 208
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 50-64 230
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 50-64 184
Geschlecht Weiblich 2412
Besiedelung mittlere Besiedlungsdichte 1224
OstWest West 729
Alter 30-49 197
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter 30-49 178
Geschlecht Männlich 2292
Besiedelung mittlere Besiedlungsdichte 1078
OstWest West 666
Alter Ü65 222
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter Ü65 76
Geschlecht Männlich 2292
Besiedelung dicht besiedelt 471
OstWest West 257
Alter 30-49 85
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 30-49 60
Geschlecht Weiblich 2412
Besiedelung dicht besiedelt 485
OstWest West 259
Alter 50-64 64
Geschlecht Männlich 2292
Besiedelung d

,za_nr,doi,version,respid,substudy,mode,splt23_1,splt23_2,eastwest,german,...,bik,xs11,wghtpew,wghtht,wghthew,wghthtew,Alter,Geschlecht,OstWest,Besiedelung
770,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",771.0,NaN,CAPI,NaN,SPLIT A,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,183.0,1.252326,1.69639,1.242308,2.107439,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3538,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3539.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,93.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
1776,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1777.0,SIMULTAN,MAIL,SPLIT B,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1-4;<50.000,10.0,1.203727,1.693259,1.192166,2.018646,Ü65,Weiblich,West,mittlere Besiedlungsdichte
3432,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3433.0,SEQUENZIELL,CAWI,SPLIT A,TNZ: MODE,ALTE BUNDESLAENDER,JA,...,ZONE 1 ;<500.000,168.0,1.207467,0.554394,1.205184,0.668147,Ü65,Weiblich,West,mittlere Besiedlungsdichte
5150,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",5151.0,NaN,CAPI,NaN,SPLIT B,ALTE BUNDESLAENDER,JA,...,ZONE 2-4;>499.999,143.0,1.252326,0.848195,1.242308,1.053719,Ü65,Weiblich,West,mittlere Besiedlungsdichte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",303.0,SIMULTAN,MAIL,SPLIT B,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,151.0,0.544816,0.845091,0.572064,0.483446,50-64,Männlich,Berlin,dicht besiedelt
677,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",678.0,SEQUENZIELL,CAWI,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,78.0,0.541072,1.579224,0.558627,0.882197,50-64,Männlich,Berlin,dicht besiedelt
3604,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",3605.0,SIMULTAN,MAIL,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,147.0,0.544816,0.563394,0.572064,0.322297,50-64,Männlich,Berlin,dicht besiedelt
1856,ALLBUS 2023,https://doi.org/10.4232/1.14387,"v1.0.0, 2024-12-03",1857.0,SEQUENZIELL,CAWI,SPLIT A,TNZ: MODE,NEUE BUNDESLAENDER,JA,...,ZONE 1 ;>499.999,147.0,0.541072,0.789612,0.558627,0.441099,50-64,Männlich,Berlin,dicht besiedelt


In [24]:
#this is a function that enables us to analyze for each for each subgroup how the respondents answered differently
#  to a given question/variable
def analyze_factor_patterns(dataset,variable_to_check):
    #we have a base dataframe that holds all results
    base_df=pd.DataFrame()
    #now we define the four features we want to use to check for changes in the respondents data in allbus
    factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]
    #we iterate through all possible combinations of these features
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #add za_nr to the list of the variable to check, as it is the unique identifier for each respondent
            baselist=[variable_to_check,"za_nr"]

            #we build also a list of the features we want to check for
            extralist=[element,element2]
            #set it to remove duplicates
            extralist=list(set(extralist))

            #and combine the two lists and filter the dataset for the columns we need
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            #remove all rows with missing values
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]

            #we also define a extralist plus which we need to group by the dataset
            #it is the name of the features we want to check for plus the variable
            # and then we count
            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            #we remove all columns that have missing values
            grouped_df=grouped_df.dropna(axis=1)
            #and then we calculate the share of the respondents for each subgroup by using the sum of the group
            grouped_df['percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')
            grouped_df['absolute'] = grouped_df['za_nr']
            
            #and we add this to the base dataframe that holds all results
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    #at the end we drop duplicates
    base_df=base_df.drop_duplicates()
    #reduce the dataset
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       'absolute','percent']
    #and return the dataset
    return base_df[base_df_cols]

In [25]:
# now we do this whole procedure for variable va03 which is for inflation
df_2023_inflation=analyze_factor_patterns(df_2023,"va03")
df_2023_inflation.sort_values(by="absolute",ascending=False)

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
9,NaN,NaN,NaN,West,AM VIERTWICHTIGSTEN,3075,39.738951
8,NaN,NaN,NaN,West,AM DRITTWICHTIGSTEN,2145,27.720341
5,NaN,NaN,Weiblich,NaN,AM VIERTWICHTIGSTEN,1875,38.699690
1,NaN,NaN,Männlich,NaN,AM VIERTWICHTIGSTEN,1853,41.278681
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,1640,38.853352
...,...,...,...,...,...,...,...
11,U30,NaN,NaN,Berlin,AM ZWEITWICHTIGSTEN,0,NaN
27,U30,NaN,NaN,Ost,AM ZWEITWICHTIGSTEN,0,NaN
26,U30,NaN,NaN,Ost,AM WICHTIGSTEN,0,NaN
25,U30,NaN,NaN,Ost,AM VIERTWICHTIGSTEN,0,NaN


In [26]:
df_2023_inflation[["Alter","Besiedelung","Geschlecht","OstWest","absolute"]]

,Alter,Besiedelung,Geschlecht,OstWest,absolute
0,30-49,NaN,NaN,NaN,714
1,30-49,NaN,NaN,NaN,925
2,30-49,NaN,NaN,NaN,426
3,30-49,NaN,NaN,NaN,684
4,50-64,NaN,NaN,NaN,767
...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,464
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,1136
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,1640
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,490


In [27]:
# now we do this whole procedure for variable va03 which is for inflation

df_2021_inflation=analyze_factor_patterns(df_2021,"va03")
df_2021_inflation

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,541,29.872998
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,843,46.548868
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,155,8.558807
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,272,15.019326
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,406,22.656250
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,218,16.515152
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,716,26.112327
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,1556,56.746900
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,109,3.975201


In [28]:
# and we merge the datasets of 2021 and 2023 based on all the features
comp_inflation=pd.merge(df_2021_inflation,df_2023_inflation,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'va03'],suffixes=["_2021","_2023"])
comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute_2021,percent_2021,absolute_2023,percent_2023
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,541,29.872998,714,25.973081
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,843,46.548868,925,33.648599
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,155,8.558807,426,15.496544
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,272,15.019326,684,24.881775
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,406,22.656250,767,28.323486
...,...,...,...,...,...,...,...,...,...
255,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,218,16.515152,464,23.211606
256,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,716,26.112327,1136,26.913054
257,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,1556,56.746900,1640,38.853352
258,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,109,3.975201,490,11.608624


In [29]:
#also we do the same for party preferences
df_2021_party=analyze_factor_patterns(df_2021,"pv01")
df_2021_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,137,6.135244
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,114,5.105240
2,30-49,NaN,NaN,NaN,CDU-CSU,477,21.361397
3,30-49,NaN,NaN,NaN,DIE GRUENEN,743,33.273623
4,30-49,NaN,NaN,NaN,DIE LINKE,143,6.403941
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,870,25.401460
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,196,5.722628
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,461,13.459854
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,537,15.678832


In [30]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2023_party

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_33510/1949256512.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,271,12.212708
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,77,3.470032
2,30-49,NaN,NaN,NaN,CDU-CSU,426,19.197837
3,30-49,NaN,NaN,NaN,DIE GRUENEN,534,24.064894
4,30-49,NaN,NaN,NaN,DIE LINKE,138,6.219018
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,706,20.091064
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,148,4.211725
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,378,10.756972
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,711,20.233352


In [31]:
#we also merge the datasets of 2021 and 2023 based on all the features for the party preferences
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
comp_party.sort_values("diff_percent")

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
110,50-64,NaN,NaN,Berlin,SPD,19,29.230769,7,9.459459,-19.771310
122,Ü65,NaN,NaN,Berlin,CDU-CSU,21,41.176471,11,21.568627,-19.607843
195,30-49,dicht besiedelt,NaN,NaN,DIE GRUENEN,428,49.365629,292,34.152047,-15.213582
355,NaN,dicht besiedelt,Männlich,NaN,DIE GRUENEN,461,38.870152,301,24.651925,-14.218227
35,30-49,NaN,Männlich,NaN,DIE GRUENEN,354,31.355182,209,17.666948,-13.688233
...,...,...,...,...,...,...,...,...,...,...
475,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE GRUENEN,0,NaN,0,NaN,NaN
476,NaN,mittlere Besiedlungsdichte,NaN,Berlin,DIE LINKE,0,NaN,0,NaN,NaN
477,NaN,mittlere Besiedlungsdichte,NaN,Berlin,FDP,0,NaN,0,NaN,NaN
478,NaN,mittlere Besiedlungsdichte,NaN,Berlin,SPD,0,NaN,0,NaN,NaN


In [32]:

#and we pivot it based on the party preferences
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
comp_party_pivot

Alter                 Besiedelung Geschlecht OstWest percent_2021  \
pv01                                                               AFD   
0      NaN                         NaN        NaN  Berlin     3.619910   
1      NaN                         NaN        NaN     Ost    11.814745   
2      NaN                         NaN        NaN    West     4.289587   
3      NaN                         NaN   Männlich     NaN     7.639074   
4      NaN                         NaN   Männlich  Berlin     6.818182   
..     ...                         ...        ...     ...          ...   
60     Ü65                         NaN   Männlich     NaN     5.900000   
61     Ü65                         NaN   Weiblich     NaN     1.906275   
62     Ü65             dicht besiedelt        NaN     NaN     3.044140   
63     Ü65            gering besiedelt        NaN     NaN     5.040323   
64     Ü65  mittlere Besiedlungsdichte        NaN     NaN     3.435805   

                                                                 ...  \
pv01 ANDERE PARTEI    CDU-CSU DIE GRUENEN  DIE LINKE        FDP  ...   
0         4.977376  22.171946   28.959276   9.049774   5.429864  ...   
1         5.387524  24.196597   13.516068  12.948960  11.342155  ...   
2         3.412534  25.912933   29.835752   5.469622  11.880083  ...   
3         3.806228  24.434389   24.700559   7.372904  12.829385  ...   
4         3.409091  15.909091   32.954545   3.409091   9.090909  ...   
..             ...        ...         ...        ...        ...  ...   
60        0.800000  31.200000   15.600000   5.200000  13.600000  ...   
61        1.985703  36.219222   17.553614   6.592534  10.484512  ...   
62        1.369863  31.050228   18.721461   3.652968   8.219178  ...   
63        2.016129  35.483871   13.709677   8.266129  11.693548  ...   
64        1.265823  35.081374   16.817360   6.329114  14.104882  ...   

     absolute_2021                      absolute_2023                        \
pv01           SPD WUERDE NICHT WAEHLEN           AFD ANDERE PARTEI CDU-CSU   
0               43                   14            17             5      39   
1              140                   80           251            38     219   
2              961                  243           480           215    1609   
3              592                  130           489           148     903   
4               20                    5            10             2      14   
..             ...                  ...           ...           ...     ...   
60             250                   27            75             8     370   
61             247                   71            57            23     426   
62             185                   38            19             6     214   
63              96                   22            46            13     193   
64             216                   38            67            12     389   

                                                            
pv01 DIE GRUENEN DIE LINKE  FDP   SPD WUERDE NICHT WAEHLEN  
0             60        39   19    40                   19  
1            105       115   67   193                   56  
2           1518       240  641  1389                  318  
3            720       193  433   767                  203  
4             19        20    7    14                    8  
..           ...       ...  ...   ...                  ...  
60           145        38   89   296                   30  
61           219        41   94   388                   52  
62            92        21   81   267                   28  
63            65        24   22   117                   17  
64           207        34   80   300                   37  

[65 rows x 36 columns]

In [33]:
# to remove the double index ot the columns, we bring them to one name in one line
new_columns=[]
for index in comp_party_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,3.619910,4.977376,22.171946,28.959276,9.049774,5.429864,...,43,14,17,5,39,60,39,19,40,19
1,NaN,NaN,NaN,Ost,11.814745,5.387524,24.196597,13.516068,12.948960,11.342155,...,140,80,251,38,219,105,115,67,193,56
2,NaN,NaN,NaN,West,4.289587,3.412534,25.912933,29.835752,5.469622,11.880083,...,961,243,480,215,1609,1518,240,641,1389,318
3,NaN,NaN,Männlich,NaN,7.639074,3.806228,24.434389,24.700559,7.372904,12.829385,...,592,130,489,148,903,720,193,433,767,203
4,NaN,NaN,Männlich,Berlin,6.818182,3.409091,15.909091,32.954545,3.409091,9.090909,...,20,5,10,2,14,19,20,7,14,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,5.900000,0.800000,31.200000,15.600000,5.200000,13.600000,...,250,27,75,8,370,145,38,89,296,30
61,Ü65,NaN,Weiblich,NaN,1.906275,1.985703,36.219222,17.553614,6.592534,10.484512,...,247,71,57,23,426,219,41,94,388,52
62,Ü65,dicht besiedelt,NaN,NaN,3.044140,1.369863,31.050228,18.721461,3.652968,8.219178,...,185,38,19,6,214,92,21,81,267,28
63,Ü65,gering besiedelt,NaN,NaN,5.040323,2.016129,35.483871,13.709677,8.266129,11.693548,...,96,22,46,13,193,65,24,22,117,17


In [34]:
#same is done for inflation
comp_inflation_pivot=comp_inflation.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='va03')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
new_columns=[]
for index in comp_inflation_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_inflation_pivot.columns=new_columns
comp_inflation_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,AM WICHTIGSTEN_percent_2023,AM ZWEITWICHTIGSTEN_percent_2023,AM DRITTWICHTIGSTEN_absolute_2021,AM VIERTWICHTIGSTEN_absolute_2021,AM WICHTIGSTEN_absolute_2021,AM ZWEITWICHTIGSTEN_absolute_2021,AM DRITTWICHTIGSTEN_absolute_2023,AM VIERTWICHTIGSTEN_absolute_2023,AM WICHTIGSTEN_absolute_2023,AM ZWEITWICHTIGSTEN_absolute_2023
0,NaN,NaN,NaN,Berlin,27.777778,48.484848,6.565657,17.171717,32.191781,34.246575,11.301370,22.260274,55,96,13,34,94,100,33,65
1,NaN,NaN,NaN,Ost,28.301887,50.943396,4.550499,16.204218,23.466258,42.407975,11.503067,22.622699,255,459,41,146,306,553,150,295
2,NaN,NaN,NaN,West,25.725726,55.675676,4.924925,13.673674,27.720341,39.738951,11.592143,20.948566,1285,2781,246,683,2145,3075,897,1621
3,NaN,NaN,Männlich,NaN,24.933687,56.730769,5.238727,13.096817,29.583426,41.278681,10.046781,19.091112,752,1711,158,395,1328,1853,451,857
4,NaN,NaN,Männlich,Berlin,24.242424,53.030303,4.545455,18.181818,37.068966,39.655172,9.482759,13.793103,16,35,3,12,43,46,11,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,21.560847,60.185185,3.439153,14.814815,28.445230,48.321555,4.770318,18.462898,163,455,26,112,322,547,54,209
61,Ü65,NaN,Weiblich,NaN,26.554268,55.742887,1.369863,16.332982,25.371207,48.030988,6.003873,20.593932,252,529,13,155,393,744,93,319
62,Ü65,dicht besiedelt,NaN,NaN,26.755218,49.905123,4.743833,18.595825,28.850856,53.789731,2.200489,15.158924,141,263,25,98,236,440,18,124
63,Ü65,gering besiedelt,NaN,NaN,22.727273,60.962567,0.267380,16.042781,22.203390,47.627119,8.983051,21.186441,85,228,1,60,131,281,53,125


In [35]:
#and then again we merge both datasets
comp_inflation_party_pivot=pd.merge(comp_inflation_pivot,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,27.777778,48.484848,6.565657,17.171717,32.191781,34.246575,...,43,14,17,5,39,60,39,19,40,19
1,NaN,NaN,NaN,Ost,28.301887,50.943396,4.550499,16.204218,23.466258,42.407975,...,140,80,251,38,219,105,115,67,193,56
2,NaN,NaN,NaN,West,25.725726,55.675676,4.924925,13.673674,27.720341,39.738951,...,961,243,480,215,1609,1518,240,641,1389,318
3,NaN,NaN,Männlich,NaN,24.933687,56.730769,5.238727,13.096817,29.583426,41.278681,...,592,130,489,148,903,720,193,433,767,203
4,NaN,NaN,Männlich,Berlin,24.242424,53.030303,4.545455,18.181818,37.068966,39.655172,...,20,5,10,2,14,19,20,7,14,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,21.560847,60.185185,3.439153,14.814815,28.445230,48.321555,...,250,27,75,8,370,145,38,89,296,30
61,Ü65,NaN,Weiblich,NaN,26.554268,55.742887,1.369863,16.332982,25.371207,48.030988,...,247,71,57,23,426,219,41,94,388,52
62,Ü65,dicht besiedelt,NaN,NaN,26.755218,49.905123,4.743833,18.595825,28.850856,53.789731,...,185,38,19,6,214,92,21,81,267,28
63,Ü65,gering besiedelt,NaN,NaN,22.727273,60.962567,0.267380,16.042781,22.203390,47.627119,...,96,22,46,13,193,65,24,22,117,17


In [36]:
comp_inflation_party_pivot.columns

Index(['Alter', 'Besiedelung', 'Geschlecht', 'OstWest',
       'AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
       'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
       'AM DRITTWICHTIGSTEN_percent_2023', 'AM VIERTWICHTIGSTEN_percent_2023',
       'AM WICHTIGSTEN_percent_2023', 'AM ZWEITWICHTIGSTEN_percent_2023',
       'AM DRITTWICHTIGSTEN_absolute_2021',
       'AM VIERTWICHTIGSTEN_absolute_2021', 'AM WICHTIGSTEN_absolute_2021',
       'AM ZWEITWICHTIGSTEN_absolute_2021',
       'AM DRITTWICHTIGSTEN_absolute_2023',
       'AM VIERTWICHTIGSTEN_absolute_2023', 'AM WICHTIGSTEN_absolute_2023',
       'AM ZWEITWICHTIGSTEN_absolute_2023', 'AFD_percent_2021',
       'ANDERE PARTEI_percent_2021', 'CDU-CSU_percent_2021',
       'DIE GRUENEN_percent_2021', 'DIE LINKE_percent_2021',
       'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_202

In [38]:
#we take in the data from the zensus that gives a share of the population for each subgroup
#but only with two features defined as the same time
zensus=pd.read_csv("json_none.csv")
zensus

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,Ost,U30,1.675804
1,NaN,gering besiedelt,NaN,U30,2.543912
2,NaN,dicht besiedelt,Ost,NaN,3.853220
3,NaN,NaN,Ost,30-49,4.559863
4,NaN,NaN,Ost,50-64,4.751534
5,NaN,NaN,Ost,Ü65,5.482466
6,NaN,gering besiedelt,Ost,NaN,5.491600
7,NaN,gering besiedelt,NaN,30-49,5.723338
8,NaN,mittlere Besiedlungsdichte,NaN,U30,5.842770
9,NaN,dicht besiedelt,NaN,U30,6.005464


In [41]:
# we merge this
comp_inflation_party_pivot_zensus=pd.merge(comp_inflation_party_pivot,zensus,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])


In [42]:
#and we filter only for the percentage results  
perc_share_df=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[~comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [43]:
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,Ost,28.301887,50.943396,4.550499,16.204218,23.466258,42.407975,...,7.561437,24.042146,3.639847,20.977011,10.057471,11.015326,6.417625,18.486590,5.363985,16.469667
1,NaN,NaN,NaN,West,25.725726,55.675676,4.924925,13.673674,27.720341,39.738951,...,3.874980,7.488300,3.354134,25.101404,23.681747,3.744150,10.000000,21.669267,4.960998,79.480028
2,NaN,NaN,Männlich,NaN,24.933687,56.730769,5.238727,13.096817,29.583426,41.278681,...,3.460208,12.681535,3.838174,23.418050,18.672199,5.005187,11.229253,19.891079,5.264523,48.259419
3,NaN,NaN,Männlich,Ost,30.210773,51.053864,4.918033,13.817330,25.538972,44.776119,...,7.604563,29.904762,3.428571,19.809524,8.190476,10.476190,6.857143,13.714286,7.619048,7.956365
4,NaN,NaN,Männlich,West,24.058660,57.788347,5.311138,12.841855,30.000000,40.769231,...,2.704423,9.947482,3.954279,24.250850,20.327464,3.645351,12.048193,21.037998,4.788384,38.393686
5,NaN,NaN,Weiblich,NaN,27.387914,52.794022,4.613385,15.204678,25.118679,38.699690,...,5.457422,6.751825,2.867570,25.130344,25.104275,5.239833,7.664234,22.288843,4.953076,51.683982
6,NaN,NaN,Weiblich,Ost,26.582278,50.843882,4.219409,18.354430,21.683310,40.370899,...,7.518797,18.111753,3.853565,22.157996,11.946050,11.560694,5.973025,23.314066,3.082852,8.513302
7,NaN,NaN,Weiblich,West,27.427184,53.519417,4.530744,14.522654,25.554435,38.760081,...,5.051151,4.979515,2.741885,25.969114,27.103687,3.844942,7.910495,22.313268,5.137094,41.086342
8,NaN,dicht besiedelt,NaN,NaN,27.312992,52.854331,5.856299,13.976378,28.998073,41.104689,...,4.130262,6.199148,2.479659,18.752422,27.237505,6.625339,8.175126,25.145293,5.385509,34.799334
9,NaN,dicht besiedelt,NaN,Ost,41.666667,43.333333,4.166667,10.833333,19.125683,33.879781,...,11.971831,19.718310,5.633803,15.492958,16.901408,14.788732,3.521127,20.422535,3.521127,3.853220


In [48]:
perc_share_df=perc_share_df.round(1)


In [49]:
#and we write this into a csv file
perc_share_df.to_csv("perc_share_df.csv")

In [50]:
# we only use the percentage data
data_with_perc=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[~comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [51]:
data_with_perc=data_with_perc.round(1)
data_with_perc["Geschlecht"]=data_with_perc["Geschlecht"].str.replace("Männlich","Maennlich")
data_with_perc["Alter"]=data_with_perc["Alter"].str.replace("Ü65","65plus")
data_with_perc["Besiedelung"]=data_with_perc["Besiedelung"].str.replace("dicht besiedelt","Stadt")
data_with_perc["Besiedelung"]=data_with_perc["Besiedelung"].str.replace("gering besiedelt","Dorf")
data_with_perc["Besiedelung"]=data_with_perc["Besiedelung"].str.replace("mittlere Besiedlungsdichte","Kleinstadt")

In [52]:
json_array=[]
for index, row in data_with_perc.iterrows():
    print(row)
    base_dict=row[["Alter","Besiedelung","Geschlecht","OstWest","Share"]].to_dict()
    base_dict["inflation"]=row[[ 'AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
        'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
        'AM DRITTWICHTIGSTEN_percent_2023', 'AM VIERTWICHTIGSTEN_percent_2023',
        'AM WICHTIGSTEN_percent_2023', 'AM ZWEITWICHTIGSTEN_percent_2023']].to_dict()
    base_dict["wahl"]=row[['AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
        'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
        'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
        'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
        'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
        'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
        'FDP_percent_2023', 'SPD_percent_2023',
        'WUERDE NICHT WAEHLEN_percent_2023']].to_dict()
    json_array.append(base_dict)
json_array

Alter                                 NaN
Besiedelung                           NaN
Geschlecht                            NaN
OstWest                               Ost
AM DRITTWICHTIGSTEN_percent_2021     28.3
AM VIERTWICHTIGSTEN_percent_2021     50.9
AM WICHTIGSTEN_percent_2021           4.6
AM ZWEITWICHTIGSTEN_percent_2021     16.2
AM DRITTWICHTIGSTEN_percent_2023     23.5
AM VIERTWICHTIGSTEN_percent_2023     42.4
AM WICHTIGSTEN_percent_2023          11.5
AM ZWEITWICHTIGSTEN_percent_2023     22.6
AFD_percent_2021                     11.8
ANDERE PARTEI_percent_2021            5.4
CDU-CSU_percent_2021                 24.2
DIE GRUENEN_percent_2021             13.5
DIE LINKE_percent_2021               12.9
FDP_percent_2021                     11.3
SPD_percent_2021                     13.2
WUERDE NICHT WAEHLEN_percent_2021     7.6
AFD_percent_2023                     24.0
ANDERE PARTEI_percent_2023            3.6
CDU-CSU_percent_2023                 21.0
DIE GRUENEN_percent_2023          

[{'Alter': nan,
  'Besiedelung': nan,
  'Geschlecht': nan,
  'OstWest': 'Ost',
  'Share': 16.5,
  'inflation': {'AM DRITTWICHTIGSTEN_percent_2021': 28.3,
   'AM VIERTWICHTIGSTEN_percent_2021': 50.9,
   'AM WICHTIGSTEN_percent_2021': 4.6,
   'AM ZWEITWICHTIGSTEN_percent_2021': 16.2,
   'AM DRITTWICHTIGSTEN_percent_2023': 23.5,
   'AM VIERTWICHTIGSTEN_percent_2023': 42.4,
   'AM WICHTIGSTEN_percent_2023': 11.5,
   'AM ZWEITWICHTIGSTEN_percent_2023': 22.6},
  'wahl': {'AFD_percent_2021': 11.8,
   'ANDERE PARTEI_percent_2021': 5.4,
   'CDU-CSU_percent_2021': 24.2,
   'DIE GRUENEN_percent_2021': 13.5,
   'DIE LINKE_percent_2021': 12.9,
   'FDP_percent_2021': 11.3,
   'SPD_percent_2021': 13.2,
   'WUERDE NICHT WAEHLEN_percent_2021': 7.6,
   'AFD_percent_2023': 24.0,
   'ANDERE PARTEI_percent_2023': 3.6,
   'CDU-CSU_percent_2023': 21.0,
   'DIE GRUENEN_percent_2023': 10.1,
   'DIE LINKE_percent_2023': 11.0,
   'FDP_percent_2023': 6.4,
   'SPD_percent_2023': 18.5,
   'WUERDE NICHT WAEHLEN_perc

In [53]:
import json

with open('data_for_viz_2.json', 'w', encoding='utf-8') as f:
    json.dump(json_array, f)